In [14]:
!pip install seaborn

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import numpy as np
import pandas as pd
import matplotlib
import seaborn as sns

### Importing the dataset.. Reading the .csv files

In [23]:
productData = pd.read_csv("dataset/product.csv",sep="\t")

In [5]:
userData = pd.read_csv("dataset/user.csv",sep="\t")

In [6]:
sessionData = pd.read_csv("dataset/session.csv",sep="\t")

In [15]:
productData.tail(10)

,location,product_ean,code,store_name
14965,Asia/Tokyo,7998806458534,JP,Minokamo
14966,America/New_York,6508748773735,US,Taunton
14967,Asia/Kolkata,6880481838958,IN,Robertsonpet
14968,Europe/Skopje,4916516663454,MK,Saraj
14969,Europe/Moscow,6841434935681,RU,Gudermes
14970,Europe/Berlin,149467746332,DE,Walsrode
14971,Asia/Yekaterinburg,3866945905056,RU,Lesnoy
14972,Asia/Kolkata,4012728908809,IN,Dalsingh Sarai
14973,Africa/Nairobi,5246696890388,KE,Lodwar
14974,Asia/Kolkata,7939158449856,IN,Osmanabad


In [ ]:
userData.head(10)

In [ ]:
sessionData.head(10)

In [10]:
productData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14975 entries, 0 to 14974
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   location     14975 non-null  object
 1   product_ean  14975 non-null  int64 
 2   code         14960 non-null  object
 3   store_name   14975 non-null  object
dtypes: int64(1), object(3)
memory usage: 468.1+ KB


In [ ]:
productData[productData.isnull().any(axis=1)]

In [27]:
productData = productData.dropna()

In [32]:
productData = productData.reset_index().drop("index",axis=1)